In [2]:
from typing import TypedDict, List, Annotated, Sequence
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate

class StoicState(TypedDict):
    messages: List[BaseMessage]
    control_analysis: str | None

def create_control_analyzer():
    # Initialize the model
    model = ChatOpenAI(model="gpt-4", temperature=0.7)
    
    # Create the control analysis prompt
    control_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a Stoic philosopher and advisor. Your task is to analyze situations and determine whether they are:
        1. Within our control (internal)
        2. Outside our control (external)
        3. Partially within our control

        Follow these Stoic principles:
        - Within our control: Our thoughts, judgments, intentions, attitudes, responses, and actions
        - Outside our control: Other people's actions, natural events, past events, and external circumstances
        - Partially within our control: Outcomes that we can influence but not completely control

        Provide your analysis in this format:
        Control Category: [Within/Outside/Partial]
        Reasoning: [Your explanation]
        Stoic Advice: [Practical advice based on Stoic principles]"""),
        ("human", "{input}")
    ])

    def analyze_control(state: StoicState) -> StoicState:
        # Get the last message
        last_message = state["messages"][-1].content
        
        # Generate the control analysis
        prompt = control_prompt.format_messages(input=last_message)
        response = model.invoke(prompt)
        
        # Update state with analysis
        state["control_analysis"] = response.content
        state["messages"].append(AIMessage(content=response.content))
        
        return state

    # Create the graph
    workflow = StateGraph(StoicState)
    
    # Add the control analysis node
    workflow.add_node("analyze_control", analyze_control)
    
    # Create edges
    workflow.set_entry_point("analyze_control")
    workflow.add_edge("analyze_control", END)
    
    # Compile the graph
    chain = workflow.compile()
    
    return chain

def analyze_situation(situation: str) -> dict:
    # Initialize the chain
    chain = create_control_analyzer()
    
    # Create initial state
    initial_state = {
        "messages": [HumanMessage(content=situation)],
        "control_analysis": None
    }
    
    # Run the analysis
    result = chain.invoke(initial_state)
    
    return result

# Break a story into a series of situations

In [6]:
# Example usage
if __name__ == "__main__":
    situations = [
        "My coworker constantly criticizes my work in front of others",
        # "I'm feeling anxious about an upcoming presentation",
        # "There's a major traffic jam and I might be late for an important meeting",
        # "I'm struggling to maintain focus during my meditation practice"
    ]
    
    for situation in situations:
        print(f"\n# Analyzing situation: {situation}")
        result = analyze_situation(situation)

        print("\n ## Analysis:")
        print(result["control_analysis"])
        str_output = result["control_analysis"]
        # first sepreate as the space backspace
        str_output = str_output.split("\n")
        # remove all the empty strings
        str_output = [x for x in str_output if x]
        # let the key be x:y y be the value
        str_output = [{x.split(":")[0]: x.split(":")[1]} for x in str_output]
        print(str_output)


# Analyzing situation: My coworker constantly criticizes my work in front of others

 ## Analysis:
Control Category: Outside

Reasoning: Your coworker's actions and remarks are external events that you cannot control. The way they choose to behave or react is dependent on their personal judgments, which fall outside of your control.

Stoic Advice: You cannot control your coworker's behavior, but you can control how you respond to it. It's important to maintain a level-headed attitude and not let their criticism affect your self-esteem or work performance. You might also consider communicating directly with your coworker about how their actions affect you, although their response remains outside your control. Remember, it's not the event itself, but your judgment about it that disturbs your tranquility.
['Control Category: Outside', "Reasoning: Your coworker's actions and remarks are external events that you cannot control. The way they choose to behave or react is dependent on their p